### Importing all the libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix


Since I comlpeted preprocessing of data I will just Load the dataset 

In [2]:
df = pd.read_csv('movies_finalized_dataset.csv')

In [3]:
df

,movieId,title,genres,imdbId,imdb_url,avg_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,https://www.imdb.com/title/tt0114709/,3.897438
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://www.imdb.com/title/tt0113497/,3.275758
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://www.imdb.com/title/tt0113228/,3.139447
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,https://www.imdb.com/title/tt0114885/,2.845331
4,5,Father of the Bride Part II (1995),Comedy,113041,https://www.imdb.com/title/tt0113041/,3.059602
...,...,...,...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama,26812510,https://www.imdb.com/title/tt26812510/,4.000000
87581,292737,Shelter in Solitude (2023),Comedy|Drama,14907358,https://www.imdb.com/title/tt14907358/,1.500000
87582,292753,Orca (2023),Drama,12388280,https://www.imdb.com/title/tt12388280/,4.000000
87583,292755,The Angry Breed (1968),Drama,64027,https://www.imdb.com/title/tt0064027/,1.000000


In [4]:
df.describe()

,movieId,imdbId,avg_rating
count,87585.000000,8.758500e+04,87585.000000
mean,157651.365519,2.792840e+06,3.005082
std,79013.402099,4.278866e+06,0.787255
min,1.000000,1.000000e+00,0.500000
25%,112657.000000,9.464200e+04,2.595652
50%,165741.000000,4.929960e+05,3.005082
75%,213203.000000,3.877296e+06,3.500000
max,292757.000000,2.908110e+07,5.000000


In [5]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title   

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     87585 non-null  int64  
 1   title       87585 non-null  object 
 2   genres      87585 non-null  object 
 3   imdbId      87585 non-null  int64  
 4   imdb_url    87585 non-null  object 
 5   avg_rating  87585 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 4.0+ MB


In [7]:
df['clean_title'] = df['title'].apply(clean_title)

In [8]:
df

,movieId,title,genres,imdbId,imdb_url,avg_rating,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,https://www.imdb.com/title/tt0114709/,3.897438,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://www.imdb.com/title/tt0113497/,3.275758,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://www.imdb.com/title/tt0113228/,3.139447,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,https://www.imdb.com/title/tt0114885/,2.845331,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,113041,https://www.imdb.com/title/tt0113041/,3.059602,Father of the Bride Part II 1995
...,...,...,...,...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama,26812510,https://www.imdb.com/title/tt26812510/,4.000000,The Monroy Affaire 2022
87581,292737,Shelter in Solitude (2023),Comedy|Drama,14907358,https://www.imdb.com/title/tt14907358/,1.500000,Shelter in Solitude 2023
87582,292753,Orca (2023),Drama,12388280,https://www.imdb.com/title/tt12388280/,4.000000,Orca 2023
87583,292755,The Angry Breed (1968),Drama,64027,https://www.imdb.com/title/tt0064027/,1.000000,The Angry Breed 1968


In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(df["clean_title"])

In [10]:
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = df.iloc[indices].iloc[::-1]
    
    return results

In [11]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [12]:

movie_id = 153933

#def find_similar_movies(movie_id):
movie = df[df["movieId"] == movie_id]

In [13]:

ratings = pd.read_csv("ratings.csv")

In [14]:

ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [15]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [16]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [17]:

similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [18]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [19]:

all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [20]:

rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [21]:
rec_percentages

,similar,all
movieId,,
153933,1.0,0.000012
68954,1.0,0.081737
135727,1.0,0.000047
122904,1.0,0.050603
117851,1.0,0.001223
...,...,...
258091,0.5,0.000006
1704,0.5,0.129392
1721,0.5,0.056914


In [22]:

rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [23]:

rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [24]:
rec_percentages.head(10).merge(df, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,imdbId,imdb_url,avg_rating,clean_title
77466,0.5,0.000006,84619.0,258031,Kaaviya Thalaivan (2014),Drama|Romance,2624704,https://www.imdb.com/title/tt2624704/,2.750000,Kaaviya Thalaivan 2014
38387,1.0,0.000012,84619.0,153933,Chatrapathi (2005),Action|Drama,843328,https://www.imdb.com/title/tt0843328/,3.735294,Chatrapathi 2005
77535,0.5,0.000006,84619.0,258261,Nee Varuvai Ena (1999),(no genres listed),2380288,https://www.imdb.com/title/tt2380288/,5.000000,Nee Varuvai Ena 1999
77536,0.5,0.000006,84619.0,258263,Chinna Thambi (1991),Drama|Romance,306645,https://www.imdb.com/title/tt0306645/,5.000000,Chinna Thambi 1991
80780,0.5,0.000006,84619.0,271164,Cat Burglar (2022),Animation|Comedy,17321170,https://www.imdb.com/title/tt17321170/,2.875000,Cat Burglar 2022
74937,0.5,0.000006,84619.0,245936,Versailles - The Dream of a King (2008),(no genres listed),1167686,https://www.imdb.com/title/tt1167686/,2.750000,Versailles The Dream of a King 2008
77475,0.5,0.000006,84619.0,258049,Aadavari Matalaku Ardhalu Verule (2007),Children|Comedy|Romance,907618,https://www.imdb.com/title/tt0907618/,3.333333,Aadavari Matalaku Ardhalu Verule 2007
77479,0.5,0.000006,84619.0,258057,Ayana (2017),(no genres listed),6891660,https://www.imdb.com/title/tt6891660/,5.000000,Ayana 2017
77505,0.5,0.000006,84619.0,258111,Idharkuthane Aasaipattai Balakumara (2013),Comedy|Romance,3219268,https://www.imdb.com/title/tt3219268/,5.000000,Idharkuthane Aasaipattai Balakumara 2013
77500,0.5,0.000006,84619.0,258101,Kis Kisko Pyaar Karoon (2015),Comedy|Romance,4302956,https://www.imdb.com/title/tt4302956/,3.000000,Kis Kisko Pyaar Karoon 2015


In [25]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [26]:

import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            recommended_movies = find_similar_movies(movie_id)  # Call your recommendation logic here
            display(recommended_movies)  # Display the recommended movies DataFrame

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='', description='Movie Title:')

Output()